# Создание новых признаков для classified_data (olist_classified_public_dataset.csv)

In [15]:
import pandas as pd
import sys, pathlib
from feature_engineering import load_data, build_feature_matrix

In [16]:
sys.path.append(str(pathlib.Path("notebooks/preprocessing").resolve()))

In [17]:
df = load_data("../../data/interim/classified_data_processed.parquet")

2025-07-15 16:48:59,926 - INFO - Loading data from ../../data/interim/classified_data_processed.parquet


In [18]:
display(df.head())

,id,order_status,order_products_value,order_freight_value,order_items_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_city,...,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied,most_voted_subclass,most_voted_class,product_category_name_english
0,1,delivered,89.99,14.38,1,2017-08-30 11:41:01,2017-08-30 11:55:08.970352,2017-09-21,2017-09-08 20:35:27.276847,belo_horizonte,...,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido,health_beauty
1,2,delivered,69.00,15.23,1,2017-09-26 09:13:36,2017-09-26 09:28:10.922048,2017-10-24,2017-09-29 21:13:04.984841,pocos_de_caldas,...,0,0,0,0,0,0,0,antes_prazo,satisfeito_com_pedido,toys
2,3,delivered,99.80,15.86,2,2018-01-15 15:50:42,2018-01-17 07:29:56.419769,2018-02-05,2018-01-23 17:51:31.134866,sao_jose_dos_campos,...,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega,garden_tools
3,4,delivered,87.00,12.74,1,2018-02-04 11:16:42,2018-02-06 05:31:50.990164,2018-03-13,2018-02-20 19:38:06.633080,ribeirao_preto,...,0,0,0,0,0,0,0,atrasado,problemas_de_entrega,computers_accessories
4,5,delivered,99.90,17.95,1,2017-12-07 11:58:42,2017-12-08 02:36:49.587515,2018-01-03,2017-12-19 22:33:18.952512,rio_de_janeiro,...,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega,bed_bath_table


## Создание временных признаков

#### На основе временных признаков можно создать новые полезные и хорошо интерпретируемые:
1. Время между оплатой заказа и подтверждением заказа на сайте (order_approved_at - order_purchase_timestamp).
2. Время между подтверждением заказа и ориентировочным времени доставки (order_estimated_delivery_date - order_aproved_at).
3. Время между фактическим времени доставки и ожидаемым (order_delivered_customer_date - order_estimated_delivery_date).
4. Время между рассылкой с просьбой оценить качество и ответом покупателя (review_answer_timestamp - review_creation_date).
5. Время между подтверждением заказа и его прибытием (order_delivered_customer_date - order_aproved_at).

In [ ]:
def new_time_features(data, feature_1, feature_2):
    return (data[feature_2] - data[feature_1]).dt.total_seconds() / 3600

In [ ]:
df['aproved-purchased'] = new_time_features(df, 'order_purchase_timestamp', 'order_aproved_at')
df['estimated_delivery-aproved'] = new_time_features(df, 'order_aproved_at', 'order_estimated_delivery_date')
df['actual_delivery-estimated_delivery'] = new_time_features(df, 'order_estimated_delivery_date', 'order_delivered_customer_date')
df['review_answer-review_creation'] = new_time_features(df, 'review_creation_date', 'review_answer_timestamp')
df['actual_delivery-aproved'] = new_time_features(df, 'order_aproved_at', 'order_delivered_customer_date')

In [ ]:
display(df[['aproved-purchased', 'estimated_delivery-aproved', 'actual_delivery-estimated_delivery', 'review_answer-review_creation', 'actual_delivery-aproved']].head())

#### Значения показывают количество часов разницы. В случае с разницей в доставке число отрицательное если заказ был доставлен раньше ориентировочного числа (скорее хорошо), и положительное, если была задержка по сравнению с ориентировочным числом (скорее плохо).

## Новые классы из votes-признаков

In [ ]:
# max_votes: максимальное число голосов в строке
# ties_mask: True там, где столбец == max_votes
max_votes  = df[votes_features].max(axis=1)
ties_mask  = df[votes_features].eq(max_votes, axis=0) # широкая матрица True/False такой же формы

display(max_votes.head())
display(ties_mask.head())

In [ ]:
# формируем новые индикаторы
new_cols = [c.replace('votes_', '') for c in votes_features] # ['satisfied', 'partial_delivery', 'not_as_anounced', ...]
df[new_cols] = ties_mask.astype(int).values # True→1, False→0

In [ ]:
display(df[new_cols].head())